In [28]:
from binance.client import Client
import time 
import pandas as pd
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


# DESCRIPTION DATASET

def description(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values
    summary['PercMissing'] = df.isnull().sum().values / df.isnull().count().values
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.iloc[0].values
    summary['Second Value'] = df.iloc[1].values
    summary['Third Value'] = df.iloc[2].values
    return summary


# COLORS

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    ORANGE = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


API_KEY = ""
SECRET_KEY = ""

with open("binanceApiKey",'r') as f :
    API_KEY = f.read()[:-1]
    
with open("binanceSecretKey",'r') as f :
    SECRET_KEY = f.read()[:-1]

client = Client(API_KEY, SECRET_KEY)

myCryptos = ["BTC","ADA","1INCH","DOT","GRT","IOTA","BTT","FIL","ETH","DASH","BNB","XLM","YFII","ATOM","LINK","PNT","OCEAN","XRP","DREP","RSR","ALGO"]
myPivots = ["USDT"]#["BTC","BUSD","USDT"]

# MIGHT NEED TO SYNCHRONISE

#print(client.get_server_time())
#print(time.localtime())


# VERIFY CREDENTIALS

# print(client.get_account())


# MARKET DEPTH
#depth = client.get_order_book(symbol='BTCUSDT')


# GET ORDERS AND CURRENT MARKET PRICE
#trades = []

#for i in range(len(myCryptos)) :
#    avg_price = client.get_avg_price(symbol='{}USDT'.format(myCryptos[i]))
#    crypto_df = pd.DataFrame(client.get_my_trades(symbol='{}USDT'.format(myCryptos[i])))
#    crypto_df["curRate"]= [float(avg_price['price']) for _ in range(len(crypto_df))]
#    trades.append(crypto_df)
#
#df = pd.concat(trades)
#df

dfs = []
for j in range(len(myPivots)):
    trades_j = []
    for i in range(len(myCryptos)) :
        if myCryptos[i] != myPivots[j] :
            symbol = '{}{}'.format(myCryptos[i],myPivots[j])
            try :
                crypto_df_j = pd.DataFrame(client.get_my_trades(symbol=symbol))
                if not crypto_df_j.empty :
                    crypto_df_j["curRate"]= [float(client.get_avg_price(symbol=symbol)['price']) for _ in range(len(crypto_df_j))]
                    trades_j.append(crypto_df_j)
            except :
                print("{} does not exist on Binance".format(symbol))

    dfs.append(pd.concat(trades_j))
    
df = dfs[0]
    


    
# start aggregated trade websocket for BNBBTC
#def process_message(msg):
#    print("message type: {}".format(msg['e']))
#    print(msg)
    # do something

#from binance.websockets import BinanceSocketManager
#bm = BinanceSocketManager(client)
#bm.start_aggtrade_socket('BTCUSDT', process_message)
#bm.start()



In [7]:
description(df)

Dataset Shape: (77, 14)


,Name,dtypes,Missing,PercMissing,Uniques,First Value,Second Value,Third Value
0,symbol,object,0,0.0,17,BTCUSDT,BTCUSDT,BTCUSDT
1,id,int64,0,0.0,77,646611875,655674186,655749341
2,orderId,int64,0,0.0,69,4840104641,4894360165,4894784541
3,orderListId,int64,0,0.0,1,-1,-1,-1
4,price,object,0,0.0,74,49182.73000000,56133.29000000,56400.00000000
5,qty,object,0,0.0,72,0.00101600,0.00035600,0.00034300
6,quoteQty,object,0,0.0,77,49.96965368,19.98345124,19.34520000
7,commission,object,0,0.0,77,0.00000102,0.00005508,0.00005190
8,commissionAsset,object,0,0.0,13,BTC,BNB,BNB
9,time,int64,0,0.0,72,1613490418240,1613835504045,1613837234487


In [29]:
reference = "USDT"
nope = "BUSDU"

def clean(df):
    #df = df.dropna()
    df = df[(df.symbol.str.match(r'.*{}.*'.format(reference))==True)]
    df = df[(df.symbol.str.match(r'.*{}'.format(nope))==False)]
    #cleanDf = pd.to_timedate(df["Date(UTC)"])
    
    cleanDf =df[["qty","price","curRate"]]
    
    cleanDf["price"] = df.price.astype(float)
    cleanDf["qty"] = df.qty.astype(float)
    cleanDf["Type"] = df.isBuyer.apply(lambda x: 0 if x else 1)
    cleanDf["symbol"] = df.symbol.apply(lambda x : x.split(reference)[0])
    
    return cleanDf


def compute_change(group):
    return pd.DataFrame({
            'totalInvested':(group[group.Type==0].price * group[group.Type==0].qty).sum(),
            'totalCashed': (group[group.Type==1].price * group[group.Type==1].qty).sum(),

            'liquidAsset': (group[group.Type==1].price * group[group.Type==1].qty).sum() -(group[group.Type==0].price * group[group.Type==0].qty).sum(),
            'cryptoAsset':group[group.Type==0].qty.sum()-group[group.Type==1].qty.sum(),
            'boughtAt': (group[group.Type==0].price * group[group.Type==0].qty).sum()/group[group.Type==0].qty.sum() ,
            'curRate':group.curRate.mean(),
    }, index=[0])


interesting_df = clean(df).groupby("symbol").apply(compute_change).reset_index()


interesting_df["BONUS"]= interesting_df.curRate*interesting_df.cryptoAsset+interesting_df.liquidAsset
bonus = interesting_df["BONUS"].sum()
if bonus >=0 :
    print(bcolors.OKGREEN+"Yes, Your curent bonus is {} USDT".format(bonus)+bcolors.ENDC)
else : 
    print(bcolors.FAIL + "Aïe, Your curent bonus is {} USDT".format(bonus)+bcolors.ENDC)
interesting_df

Yes, Your curent bonus is 262.1950772364185 USDT


,symbol,level_1,totalInvested,totalCashed,liquidAsset,cryptoAsset,boughtAt,curRate,BONUS
0,1INCH,0,99.928188,52.377600,-47.550588,9.940000,5.031631,4.640889,-1.420147
1,ADA,0,1167.065205,1275.090869,108.025664,101.000000,1.021680,1.259587,235.243912
2,ALGO,0,99.983755,111.721800,11.738045,0.000000,1.138249,1.260384,11.738045
3,ATOM,0,49.899380,48.410000,-1.489380,0.000000,24.223000,22.909342,-1.489380
4,BNB,0,296.169500,147.728275,-148.441225,0.504000,295.283649,264.006409,-15.381995
5,BTC,0,69.953105,19.345200,-50.607905,0.001029,50986.228076,57960.208409,9.033150
6,BTT,0,10.999615,11.430089,0.430474,10.000000,0.001182,0.001628,0.446752
7,DASH,0,109.996108,69.999811,-39.996297,0.228750,231.595132,228.011967,12.161440
8,DOT,0,49.941056,40.757330,-9.183726,0.510000,28.375600,35.933406,9.142311
9,DREP,0,99.988272,77.233310,-22.754962,2520.000000,0.015366,0.019346,25.995849


In [41]:
# FUTURES

myFuturesCryptos = ["ENJ","ADA","ETH"]
myFuturesPivots = ["USDT"]


futures_dfs = []
for j in range(len(myFuturesPivots)):
    trades_j = []
    for i in range(len(myFuturesCryptos)) :
        if myFuturesCryptos[i] != myFuturesPivots[j] :
            symbol = '{}{}'.format(myFuturesCryptos[i],myFuturesPivots[j])
            try :
                crypto_df_j = pd.DataFrame(client.futures_account_trades(symbol=symbol))
                if not crypto_df_j.empty :
                    crypto_df_j["curRate"]= [float(client.get_avg_price(symbol=symbol)['price']) for _ in range(len(crypto_df_j))]
                    trades_j.append(crypto_df_j)
            except (RuntimeError, TypeError, NameError):
                print(RuntimeError)
                print("{} does not exist on Binance".format(symbol))

    futures_dfs.append(pd.concat(trades_j))
    
future_df = futures_dfs[0]
future_df.qty = future_df.qty.astype(float)
future_df.realizedPnl = future_df.realizedPnl.astype(float)

future_bonus = future_df.realizedPnl.astype(float).sum()
if future_bonus >=0 :
    print(bcolors.OKGREEN+"Yes, Your curent future bonus is {} USDT".format(future_bonus)+bcolors.ENDC)
else : 
    print(bcolors.FAIL + "Aïe, Your curent future bonus is {} USDT".format(future_bonus)+bcolors.ENDC)
    
print(bcolors.OKBLUE + "\nYour total bonus is {} USDT, without including order fees, for {} spot orders and {} future orders\n".format(future_bonus+bonus,df.shape[0],future_df.shape[0])+bcolors.ENDC)

# Note that future orders are not correct 

future_df.groupby("symbol").sum()[["qty","realizedPnl"]]

Yes, Your curent future bonus is 128.11310992999998 USDT

Your total bonus is 390.3081871664185 USDT, without including order fees, for 101 spot orders and 95 future orders



,qty,realizedPnl
symbol,,
ADAUSDT,11505.0,92.58870
ENJUSDT,5200.0,35.14441
ETHUSDT,2.0,0.38000


In [42]:
future_df

,symbol,id,orderId,side,price,qty,realizedPnl,marginAsset,quoteQty,commission,commissionAsset,time,positionSide,maker,buyer,curRate
0,ENJUSDT,26881030,2913706569,BUY,2.35269,300.000,0.000000,USDT,705.80700,0.14116140,USDT,1615978130854,BOTH,True,True,2.640295
1,ENJUSDT,26882028,2913728454,SELL,2.36479,148.000,1.790800,USDT,349.98892,0.13999556,USDT,1615978219662,BOTH,False,False,2.640295
2,ENJUSDT,26882029,2913728454,SELL,2.36401,152.000,1.720640,USDT,359.32952,0.14373180,USDT,1615978219662,BOTH,False,False,2.640293
3,ENJUSDT,26903908,2914219277,BUY,2.32643,9.000,0.000000,USDT,20.93787,0.00837514,USDT,1615980305929,BOTH,False,True,2.640293
4,ENJUSDT,26903909,2914219277,BUY,2.32644,80.000,0.000000,USDT,186.11520,0.07444608,USDT,1615980305929,BOTH,False,True,2.640293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,ADAUSDT,137905474,11243068170,BUY,1.43651,3327.000,74.576891,USDT,4779.26877,1.91170750,USDT,1616082225193,BOTH,False,True,1.240924
0,ETHUSDT,411261257,8389765494236701331,SELL,1819.69,1.000,0.000000,USDT,1819.69000,0.72787600,USDT,1616063578076,BOTH,False,False,1788.546149
1,ETHUSDT,411264644,8389765494236785260,BUY,1819.31,0.090,0.034200,USDT,163.73790,0.06549516,USDT,1616063833713,BOTH,False,True,1788.545379
2,ETHUSDT,411264645,8389765494236785260,BUY,1819.31,0.004,0.001520,USDT,7.27724,0.00291089,USDT,1616063833713,BOTH,False,True,1788.333857
